# Comparing a USGSCSM and ISIS camera for Cassini ISS

In [1]:
import pvl
import numpy as np
import os
import pandas as pd
import knoten
import csmapi

os.environ['ISISROOT'] = '/usgs/pkgs/isis3.8.0_RC1/install'
from pysis import isis
from pysis.exceptions import ProcessError

## Make a CSM sensor model
Requires N1702360370_1.LBL and N1702360370_1.IMG in data directory

In [12]:
fileName = 'data/N1702360370_1.LBL'
camera = knoten.csm.create_csm(fileName)

## Ingest the image and spiceinit

In [3]:
# Set the output location of the resulting .cub
cub_loc = os.path.splitext(fileName)[0] + '.cub'

try: 
    isis.ciss2isis(from_=fileName, to=cub_loc)
except ProcessError as e:
    print(e.stderr)

try:
    isis.spiceinit(from_=cub_loc, shape='ellipsoid')
except ProcessError as e:
    print(e.stderr)

## Define a function that compares ISIS and USGSCSM pixels

In [4]:
def check_pixel(camera, cub, line, sample):
    """Compares ISIS and USGSCSM pixel.
    
    Takes an image coordinate, projects it to a ground point using ISIS, then projects
    the result back into an image coordinate using USGSCSM and computes the difference
    between image coordinates.
    """
    output = isis.campt(from_=cub, line=line, sample=sample)
    pvl_output = pvl.loads(output)
    bodyfixed = pvl_output['GroundPoint']['BodyFixedCoordinate']
    bodyfixed = np.asarray(bodyfixed.value) * 1000
    image_coord = camera.groundToImage(csmapi.EcefCoord(*bodyfixed))
    # (.5,.5) in CSM == (1,1) in ISIS, so we have to subtract (.5,.5) from the ISIS pixels
    line_diff = line - image_coord.line - .5
    sample_diff = sample - image_coord.samp - .5
    return line_diff, sample_diff

## Get the total number of lines / samples

In [5]:
isis_label = pvl.load(cub_loc)
n_samples = isis_label['IsisCube']['Core']['Dimensions']['Samples']
n_lines = isis_label['IsisCube']['Core']['Dimensions']['Lines']

## Compare top left, top right, bottom left, bottom right, and center pixels using check_pixel

In [13]:
pixels_dict = {'line' : [1,1,n_lines, n_lines, n_lines/2],
               'sample' : [1, n_samples, 1, n_samples, n_samples/2]}

pixels_df = pd.DataFrame.from_dict(pixels_dict)
pixels_df['line_diff'] = np.NaN
pixels_df['sample_diff'] = np.NaN

for idx, row in pixels_df.iterrows():
    pixels_df.iloc[idx]['line_diff'], pixels_df.iloc[idx]['sample_diff'] = check_pixel(camera, cub_loc, row['line'], row['sample'])
    
pixels_df

,line,sample,line_diff,sample_diff
0,1.0,1.0,0.153039,0.906085
1,1.0,1024.0,0.142231,0.326977
2,1024.0,1.0,-0.425122,0.915083
3,1024.0,1024.0,-0.435769,0.335980
4,512.0,512.0,-0.142021,0.620648
